In [ ]:
# @title Environment Setup
import os, re, subprocess, sys

def run(cmd):
    print('+', ' '.join(cmd))
    subprocess.run(cmd, check=True)

try:
    run(['apt-get', 'update'])
    run(['apt-get', 'install', '-y', 'ffmpeg'])
except Exception as e:
    print('Skipping apt-get install:', e)


gpu = subprocess.run(['bash','-lc','nvidia-smi'], capture_output=True, text=True)
has_gpu = gpu.returncode == 0
index = os.environ.get('PYTORCH_INDEX_URL')
if not index and has_gpu:
    m = re.search(r'CUDA Version: (\d+)\.(\d+)', gpu.stdout or gpu.stderr)
    if m:
        index = f"https://download.pytorch.org/whl/cu{m.group(1)}{m.group(2)}"
if not index:
    index = "https://download.pytorch.org/whl/cpu"
print('Using PyTorch index:', index)
try:
    run([sys.executable, '-m', 'pip', 'install', f'--index-url={index}', 'torch==2.3.0', 'torchvision==0.18.0', 'torchaudio==2.3.0'])
except subprocess.CalledProcessError:
    fallback = "https://download.pytorch.org/whl/cpu"
    print('PyTorch install failed, falling back to', fallback)
    run([sys.executable, '-m', 'pip', 'install', f'--index-url={fallback}', 'torch==2.3.0', 'torchvision==0.18.0', 'torchaudio==2.3.0'])

req = 'requirements-colab-gpu.txt' if has_gpu else 'requirements-colab-cpu.txt'
run([sys.executable, '-m', 'pip', 'install', '-r', req])
subprocess.run(['ffmpeg', '-version'])
import torch, soxr
print('torch', torch.__version__)
print('CUDA available:', torch.cuda.is_available())


In [ ]:
# @title Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')


In [ ]:
# @title Upload stems to Drive
# Upload four WAV files named vocals.wav, drums.wav, bass.wav and other.wav
from google.colab import files
import os
INPUT_DIR = '/content/drive/MyDrive/AutoMix/input'  # @param {type:"string"}
os.makedirs(INPUT_DIR, exist_ok=True)
uploaded = files.upload()
for name, data in uploaded.items():
    with open(os.path.join(INPUT_DIR, name), 'wb') as f:
        f.write(data)
print('Uploaded files:', list(uploaded.keys()))


In [ ]:
# @title Run pipeline
import os, sys, subprocess
INPUT_DIR = '/content/drive/MyDrive/AutoMix/input'  # @param {type:"string"}
OUTPUT_DIR = '/content/drive/MyDrive/AutoMix/output'  # @param {type:"string"}
os.makedirs(OUTPUT_DIR, exist_ok=True)
cmd = [sys.executable, 'scripts/pipeline_gdrive.py', '--input', INPUT_DIR, '--output', OUTPUT_DIR]
subprocess.run(cmd, check=True)
print('Output files:', os.listdir(OUTPUT_DIR))


In [ ]:
# @title Play mixed result
import os
from IPython.display import Audio
OUTPUT_DIR = '/content/drive/MyDrive/AutoMix/output'  # @param {type:"string"}
audio_path = os.path.join(OUTPUT_DIR, 'mix.wav')
if os.path.exists(audio_path):
    display(Audio(filename=audio_path))
else:
    print('mix.wav not found')
